In [77]:
#!value --name strategyGuide --from-file strategy.txt

In [78]:
using System;
using LanguageExt;

readonly record struct OpponentChoice(string Value)
{
   public static implicit operator string(OpponentChoice choice) => choice.Value;
   public static implicit operator OpponentChoice(string value) => new(value);
}

readonly record struct PredictedChoice(string Value)
{
   public static implicit operator string(PredictedChoice choice) => choice.Value;
   public static implicit operator PredictedChoice(string value) => new(value);
}

readonly record struct StrategyChoice(string Value)
{
   public static implicit operator string(StrategyChoice choice) => choice.Value;
   public static implicit operator StrategyChoice(string value) => new(value);
}

readonly record struct Round(OpponentChoice opponent, PredictedChoice prediction);

var scorePrediction = (PredictedChoice p) => p.Value switch
{
    "X" => 1,
    "Y" => 2,
    "Z" => 3,
    _ => -1
};

var win = (OpponentChoice o) => (PredictedChoice p) => o.Value switch
{
    "A" => "Y",
    "B" => "Z",
    "C" => "X",
    _ => "?"
};

var lose = (OpponentChoice o) => (PredictedChoice p) => o.Value switch
{
    "A" => "Z",
    "B" => "X",
    "C" => "Y",
    _ => "?"
};

var draw = (OpponentChoice o) => (PredictedChoice p) => o.Value;

var scoreHand = (OpponentChoice o) => (PredictedChoice p) => o.Value switch
{
    "A" when p.Value == "X" => 3,
    "B" when p.Value == "Y" => 3,
    "C" when p.Value == "Z" => 3,
    "A" when p.Value == "Z" => 0,
    "B" when p.Value == "X" => 0,
    "C" when p.Value == "Y" => 0,
    "C" when p.Value == "X" => 6,
    "A" when p.Value == "Y" => 6,
    "B" when p.Value == "Z" => 6,
    _ => -1
};

var totalScoreForRound = (OpponentChoice o) => (PredictedChoice p) =>
    scorePrediction(p) + scoreHand(o)(p);

var scoreGame = (string strategyGuide) =>
    strategyGuide
    .Split(Environment.NewLine)
    .Select(s => {
        OpponentChoice o = s.Split(' ').First();
        PredictedChoice p = s.Split(' ').Skip(1).First();
        return new Round(o, p);
    })
    .Select(r => totalScoreForRound(r.opponent)(r.prediction))
    .Sum();

In [79]:
// proof on sample data

var sampleStrategy =
"""
A Y
B X
C Z
""";

var totalScore = scoreGame(sampleStrategy);
totalScore == 15

Error: (10,18): error CS0149: Method name expected

In [80]:
#!share strategyGuide --from value

// part 1
var totalScore = scoreGame(strategyGuide);
totalScore

Error: (2,18): error CS0149: Method name expected

In [81]:
// part 2 proof on sample data

var totalScore = scoreGame(sampleStrategy);
totalScore == 12

Error: (3,18): error CS0149: Method name expected